## Test script for app

This notebook includes all of the test code I ran to try out different query methods, including: 
- Sentiment analysis for lyrics: use VADER and HuggingFace pipeline, results may be usable for English lyrics, but was not optimal for other languages
- TF-IDF to query for top keywords: use [text2text library](https://github.com/artitw/text2text#tf-idf)
- Summarize lyrics and mood to use as query: use NLTK

### Spotify and Genius Authentication

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

# Get Spotify app credentials
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIFY_REDIRECT_URI")
scope = "user-library-read playlist-read-private user-top-read"

# Authenticate 
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)

oauth_manager = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope
)

sp = spotipy.Spotify(oauth_manager=oauth_manager)
sp.current_user()

{'display_name': 'rile',
 'external_urls': {'spotify': 'https://open.spotify.com/user/4ssmxat4kvsarq280tilxd9cw'},
 'followers': {'href': None, 'total': 15},
 'href': 'https://api.spotify.com/v1/users/4ssmxat4kvsarq280tilxd9cw',
 'id': '4ssmxat4kvsarq280tilxd9cw',
 'images': [{'height': 300,
   'url': 'https://i.scdn.co/image/ab6775700000ee85b075d0a3febdc0142cda438f',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67757000003b82b075d0a3febdc0142cda438f',
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:4ssmxat4kvsarq280tilxd9cw'}

### Fetch Spotify songs

In [2]:
top_songs = sp.current_user_top_tracks(limit=50)
user_songs = []
for song in top_songs['items']:
    user_songs.append({
        'id': song['id'],
        'title': song['name'],
        'artist': song['artists'][0]['name'],
        'album': song['album']['name'],
        'release_date': song['album']['release_date'],
        'popularity': song['popularity']
    })

In [3]:
import pandas as pd
user_songs = pd.DataFrame(user_songs)
user_songs

,id,title,artist,album,release_date,popularity
0,20jc2qrUesWnyDYnWq0XW8,Nếu Ngày Ấy,SOOBIN,Nếu Ngày Ấy,2019-07-22,55
1,5OrpSDeWfWr33q0a5Ydfhr,MẸ YÊU CON,Cường Seven,Anh Trai Vượt Ngàn Chông Gai 2024 - Chung Kết,2024-10-19,53
2,6Dh0MoFwSqqyK2V6172eol,Buông,Bằng Kiều,Anh Trai Vượt Ngàn Chông Gai 2024 - Công Diễn 3,2024-08-24,38
3,2GxPAMGvsITmJJiIwHG8yS,Người Nào Đó,JustaTee,Người Nào Đó,2013-05-12,44
4,1DGeOHzwCxwJBiitcnUz0B,không biết nên vui hay buồn (feat. Táo),Bảo Anh,không biết nên vui hay buồn,2023-05-21,36
5,49kZuPsl8pXRJJGHmwRuW7,Bầu Trời Mới,Da LAB,Bầu Trời Mới,2024-10-02,58
6,5H47fXqOvbQsefEm3e0B80,Thế Giới Kẻ Thất Tình,Mai Tiến Dũng,Thế Giới Kẻ Thất Tình,2024-05-10,18
7,6E3P4uOfKHYXJLK6Q3s0Tj,Cơn Mưa Cuối,JustaTee,Cơn Mưa Cuối,2016-11-03,50
8,4fj4Jn4nw8fGX8xdbCntJf,Sau Cơn Mưa,CoolKid,Sau Cơn Mưa,2023-07-28,66
9,7yfcQynap4xcNqnydaFw83,Ai Mà Biết Được (feat. tlinh),SOOBIN,BẬT NÓ LÊN,2024-06-25,53


### Match songs with lyrics

In [4]:
import lyricsgenius
gen_client_access_token = os.getenv("GENIUS_CLIENT_TOKEN")
genius = lyricsgenius.Genius(gen_client_access_token, sleep_time = 1)
list_lyrics = []

for i, song in user_songs.iterrows():
    title = song['title']
    artist = song['artist']
    lyrics = genius.search_song(title, artist)
    if lyrics:
        list_lyrics.append({
            'title': title,
            'lyrics': lyrics.lyrics
        })
    else:
        lyrics = genius.search_song(title)
        if lyrics:
            list_lyrics.append({
                'title': title,
                'lyrics': lyrics.lyrics
            })

Searching for "Nếu Ngày Ấy" by SOOBIN...
No results found for: 'Nếu Ngày Ấy SOOBIN'
Searching for "Nếu Ngày Ấy"...
Done.
Searching for "MẸ YÊU CON" by Cường Seven...
No results found for: 'MẸ YÊU CON Cường Seven'
Searching for "MẸ YÊU CON"...
Done.
Searching for "Buông" by Bằng Kiều...
No results found for: 'Buông Bằng Kiều'
Searching for "Buông"...
Done.
Searching for "Người Nào Đó" by JustaTee...
No results found for: 'Người Nào Đó JustaTee'
Searching for "Người Nào Đó"...
No results found for: 'Người Nào Đó'
Searching for "không biết nên vui hay buồn (feat. Táo)" by Bảo Anh...
Done.
Searching for "Bầu Trời Mới" by Da LAB...
Done.
Searching for "Thế Giới Kẻ Thất Tình" by Mai Tiến Dũng...
Done.
Searching for "Cơn Mưa Cuối" by JustaTee...
Done.
Searching for "Sau Cơn Mưa" by CoolKid...
No results found for: 'Sau Cơn Mưa CoolKid'
Searching for "Sau Cơn Mưa"...
Done.
Searching for "Ai Mà Biết Được (feat. tlinh)" by SOOBIN...
Done.
Searching for "NÉT" by Cường Seven...
No results found fo

Clean and tokenize lyrics

In [5]:
import random
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

# nltk.download('stopwords')
# nltk.download('wordnet')   
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')   

def clean_lyrics(df, column):
    """
    Cleans the words without importance and fix the format of the  dataframe's column lyrics 
    Args:
        df (DataFrame): df containing song information
        column (str): column to clean
    Returns:
        df (DataFrame): DataFrame containing the cleaned lyrics
    """
    df[column] = df[column].str.lower()
    # remove section marker
    df[column] = df[column].str.replace(r"(verse\s?\d*|chorus|bridge|outro|intro)", "", regex=True)
    df[column] = df[column].str.replace(r"(instrumental|guitar|solo)", "", regex=True) 
    df[column] = df[column].str.replace(r"\[.*?\]", "", regex=True)
    # remove new line
    df[column] = df[column].str.replace(r"\n", ". ", regex=True)
    # remove special characters
    df[column] = df[column].str.replace(r"[^\w\d'\s.]+", "", regex=True)
    df[column] = df[column].str.strip()

    return df

In [6]:
# Convert list_lyrics to DataFrame
list_lyrics_df = pd.DataFrame(list_lyrics)

# Clean and tokenize lyrics
list_lyrics_df = clean_lyrics(list_lyrics_df, 'lyrics')
list_lyrics_df

,title,lyrics
0,Nếu Ngày Ấy,đã từng hạnh phúc. đã từng đậm sâu. đã từng cù...
1,MẸ YÊU CON,. . . à à ời. à à ơi. cái cò đi đón cơn mưa. t...
2,Buông,. . . uh uh uh uh uh. la la la la la. . . buon...
3,không biết nên vui hay buồn (feat. Táo),. . đâu ai hay ký ức xưa ngỡ như mây. bóng dán...
4,Bầu Trời Mới,. . . thức giấc mỗi ngày là khi màn đêm vừa ké...
5,Thế Giới Kẻ Thất Tình,. sau cơn mưa đêm qua. ký ức bỗng ướt nhoà. nh...
6,Cơn Mưa Cuối,lâu nay ta quên nhau chưa. lời ca anh viết vẫn...
7,Sau Cơn Mưa,. . . vài người nói với em. lòng thủy chung có...
8,Ai Mà Biết Được (feat. tlinh),. . . yeah em đang giấu gì. chẳng buồn nói lí ...
9,NÉT,. ree ouch hey hm la. ree ouch hey hm la. ree ...


### Sentiment analysis 

Using NLTK VADER

In [7]:
from nltk.sentiment import vader
# nltk.download('vader_lexicon')

negative = []
neutral = []
positive = []
compound = []

analyzer = vader.SentimentIntensityAnalyzer()

for text in list_lyrics_df['lyrics']:
    scores = analyzer.polarity_scores(text)
    negative.append(scores['neg'])
    neutral.append(scores['neu'])
    positive.append(scores['pos'])
    compound.append(scores['compound'])

list_lyrics_df['negative'] = negative
list_lyrics_df['neutral'] = neutral
list_lyrics_df['positive'] = positive
list_lyrics_df['compound'] = compound

In [8]:
# get mean sentiment score for dataset
list_sentiment = list_lyrics_df[['negative', 'neutral', 'positive', 'compound']].mean(axis=1)
list_sentiment.mean()

np.float64(0.24540437499999998)

Sentiment analysis using [this model](tabularisai/multilingual-sentiment-analysis) from HuggingFace:
- Better than VADER: 
  - Less neutral results
  - Language support
- VADER: 
  - Excellent sentiment for English
  - Tend to lean toward neutral?

In [9]:
# Example
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# def predict_sentiment(text):
#         inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
#         sentiment_map = {
#             0: "sombre",
#             1: "sad",
#             2: "neutral",
#             3: "happy",
#             4: "ecstatic"
#         }
#         return [sentiment_map[p] for p in torch.argmax(probabilities, dim=-1).tolist()] 

def predict_sentiment(texts):
    # sentiment_map = {0: "Somber", 1: "Sad", 2: "Neutral", 3: "Happy", 4: "Estactic"}
    pipe = pipeline(task="sentiment-analysis", model=model_name)
    sentiments = pipe(texts)
    labels = [sentiment['label'] for sentiment in sentiments]
    # return [sentiment_map[int(p)] for p in labels]
    return labels
    
# texts = [
#     # English
#     "I absolutely love the new design of this app!", "The customer service was disappointing.", "The weather is fine, nothing special.",
#     # Chinese
#     "这家餐厅的菜味道非常棒！", "我对他的回答很失望。", "天气今天一般。",
#     # Spanish
#     "¡Me encanta cómo quedó la decoración!", "El servicio fue terrible y muy lento.", "El libro estuvo más o menos.",
#     # Arabic
#     "الخدمة في هذا الفندق رائعة جدًا!", "لم يعجبني الطعام في هذا المطعم.", "كانت الرحلة عادية。",
#     # Ukrainian
#     "Мені дуже сподобалася ця вистава!", "Обслуговування було жахливим.", "Книга була посередньою。",
#     # Hindi
#     "यह जगह सच में अद्भुत है!", "यह अनुभव बहुत खराब था।", "फिल्म ठीक-ठाक थी।",
#     # Bengali
#     "এখানকার পরিবেশ অসাধারণ!", "সেবার মান একেবারেই খারাপ।", "খাবারটা মোটামুটি ছিল।",
#     # Portuguese
#     "Este livro é fantástico! Eu aprendi muitas coisas novas e inspiradoras.", 
#     "Não gostei do produto, veio quebrado.", "O filme foi ok, nada de especial.",
#     # Japanese
#     "このレストランの料理は本当に美味しいです！", "このホテルのサービスはがっかりしました。", "天気はまあまあです。",
#     # Russian
#     "Я в восторге от этого нового гаджета!", "Этот сервис оставил у меня только разочарование.", "Встреча была обычной, ничего особенного.",
#     # French
#     "J'adore ce restaurant, c'est excellent !", "L'attente était trop longue et frustrante.", "Le film était moyen, sans plus.",
#     # Turkish
#     "Bu otelin manzarasına bayıldım!", "Ürün tam bir hayal kırıklığıydı.", "Konser fena değildi, ortalamaydı.",
#     # Italian
#     "Adoro questo posto, è fantastico!", "Il servizio clienti è stato pessimo.", "La cena era nella media.",
#     # Polish
#     "Uwielbiam tę restaurację, jedzenie jest świetne!", "Obsługa klienta była rozczarowująca.", "Pogoda jest w porządku, nic szczególnego.",
#     # Tagalog
#     "Ang ganda ng lugar na ito, sobrang aliwalas!", "Hindi maganda ang serbisyo nila dito.", "Maayos lang ang palabas, walang espesyal.",
#     # Dutch
#     "Ik ben echt blij met mijn nieuwe aankoop!", "De klantenservice was echt slecht.", "De presentatie was gewoon oké, niet bijzonder.",
#     # Malay
#     "Saya suka makanan di sini, sangat sedap!", "Pengalaman ini sangat mengecewakan.", "Hari ini cuacanya biasa sahaja.",
#     # Korean
#     "이 가게의 케이크는 정말 맛있어요!", "서비스가 너무 별로였어요.", "날씨가 그저 그렇네요.",
#     # Swiss German
#     "Ich find dä Service i de Beiz mega guet!", "Däs Esä het mir nöd gfalle.", "D Wätter hüt isch so naja.",
#     # Vietnamese
#     "Tôi thích cách trang trí mới của quán!", "Dịch vụ khách hàng thì thất vọng.", "Bộ phim này tạm ổn, không gì đặc biệt."
# ]

# for text, sentiment in zip(texts, predict_sentiment(texts)):
#     print(f"Text: {text}\nSentiment: {sentiment}\n")

c:\Users\bi020\Downloads\data\techprep\SpotiFine\spotify\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
sentiments = []
for text in list_lyrics_df['lyrics']:
    sentiment = (predict_sentiment([text[:512]])[0])
    sentiments.append(sentiment)

list_lyrics_df['sentiment'] = sentiments

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


### Get embedding for similarity search
[Embedding model: ](sentence-transformers/distiluse-base-multilingual-cased)

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

sentences = ["This is an example", "Đây là một ví dụ", "C'est un exemple", "这是一个例子"]

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased')
embeddings = model.encode(sentences)

for i, sentence in enumerate(sentences):
    print(f"Sentences: {sentence}")
    for j, sentence in enumerate(sentences):
        print(str(j) + f": {cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]:.4f}")

Sentences: This is an example
0: 1.0000
1: 0.9409
2: 0.9583
3: 0.9228
Sentences: Đây là một ví dụ
0: 0.9409
1: 1.0000
2: 0.9728
3: 0.9859
Sentences: C'est un exemple
0: 0.9583
1: 0.9728
2: 1.0000
3: 0.9804
Sentences: 这是一个例子
0: 0.9228
1: 0.9859
2: 0.9804
3: 1.0000


In [12]:
print(embeddings[0].shape)

(512,)


In [13]:
lyrics_embed = []
for text in list_lyrics_df['lyrics']:
    lyrics_embed.append(model.encode(text))

list_lyrics_df['lyrics_embed'] = lyrics_embed

# TF-IDF to get top keywords

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=50)
all_lyrics = "".join(list_lyrics_df['lyrics'])
lyrics_tfidf = vectorizer.fit_transform([all_lyrics])
query = vectorizer.get_feature_names_out()
query.tolist()

['ai',
 'anh',
 'bao',
 'biết',
 'bên',
 'cho',
 'chẳng',
 'chỉ',
 'còn',
 'có',
 'em',
 'giờ',
 'khi',
 'không',
 'luôn',
 'là',
 'lúc',
 'lại',
 'muốn',
 'mình',
 'mưa',
 'mắt',
 'một',
 'ngày',
 'người',
 'nhau',
 'như',
 'nhưng',
 'nhớ',
 'những',
 'này',
 'nơi',
 'oh',
 'rồi',
 'sẽ',
 'ta',
 'thật',
 'tim',
 'trong',
 'trời',
 'tình',
 'từng',
 'và',
 'vì',
 'vẫn',
 'yêu',
 'đi',
 'đã',
 'được',
 'để']

This is a library that supports multiple languages, including Vietnamese and Korean

In [15]:
# example: https://colab.research.google.com/drive/1RaWj5SqWvyC2SsCTGg8IAVcl9G5hOB50?usp=sharing
import text2text as t2t

top_kw = t2t.Tfidfer().transform(all_lyrics)

kw_query = []
for kw in top_kw:
    kw_query += list(kw.keys())

kw_query

['▁đã',
 '▁từng',
 '▁hạnh',
 '▁phúc',
 '.',
 '▁đ',
 'ậm',
 '▁sâu',
 '▁cùng',
 '▁nhau',
 '▁qua',
 'ớ',
 'n',
 '▁đau',
 '▁trách',
 '▁anh',
 '▁l',
 'ầm',
 '▁lỗi',
 '▁bao',
 '▁lần',
 '▁sai',
 '▁để',
 '▁cho',
 '▁giờ',
 '▁đây',
 '▁người',
 '▁bước',
 '▁đi',
 '▁.',
 '▁mưa',
 '▁rơi',
 '▁',
 'ướ',
 't',
 '▁vai',
 '▁cũng',
 '▁là',
 '▁lúc',
 '▁tình',
 '▁ta',
 '▁ph',
 'ai',
 '▁ai',
 'ơi',
 '▁nếu',
 '▁xin',
 '▁đừng',
 '▁mang',
 '▁the',
 '8',
 'o',
 '▁những',
 '▁nghĩ',
 '▁suy',
 '▁kí',
 'ức',
 '▁x',
 'ưa',
 '▁ngày',
 '▁nắng',
 '▁d',
 'ãi',
 '▁dầu',
 '▁ước',
 '▁như',
 '▁đầu',
 '▁yêu',
 '▁mà',
 '▁ch',
 'ẳ',
 'g',
 '▁th',
 '▁được',
 '▁quay',
 '▁ngược',
 '▁thời',
 '▁gian',
 '▁sẽ',
 '▁chẳng',
 'ở',
 '▁than',
 '▁bỏ',
 '▁hết',
 '▁công',
 '▁việc',
 '▁dang',
 '▁ô',
 'm',
 '▁lấy',
 '▁em',
 '▁vô',
 '▁và',
 '▁đến',
 '▁nơi',
 '▁mình',
 '▁chưa',
 '▁tới',
 '▁thu',
 '▁lá',
 '▁chơi',
 '▁v',
 '▁rồi',
 '▁đông',
 '▁có',
 '▁trên',
 'à',
 'ạt',
 '▁phố',
 '▁mùa',
 '▁xu',
 'ân',
 '▁hát',
 '▁ca',
 '▁quên',
 '▁mọi',
 '▁khổ',


### Get recommendations using spotify API search function

In [16]:
# take random 50 top keywords
kw_query = random.sample(kw_query, 50)
search_recs = sp.search(q="".join(kw_query), type="track", limit=30)
recs = []
for track in search_recs['tracks']['items']:
    recs.append({
        'name': track['name'],
        'artist': track['artists'][0]['name']
    })
pd.DataFrame(recs)

,name,artist
0,Lửng Lơ - Remix,Masew
1,Lửng Lơ (Remix) - Beat,1 9 6 7
2,Bác đang cùng chúng cháu hành quân,Tốp ca
3,Rất Lâu Rồi Mới Khóc - Mike.N Remix,BMZ
4,Lạnh Lẽo - Thái Hoàng Remix,Thái Hoàng
5,Rửa Mặt Như Mèo,Mầm Chồi Lá
6,Lửng Lơ - Lofi,Masew
7,Em,Kidz
8,Nụ Hồng Mong Manh,Kidz
9,Hồ Sơ Tuyệt Mật,Kidz


### Summarization 

Instead of using top keywords, use a brief query that acts as a summary for the playlist

In [17]:
def text_summarizer(text, num_sen = 1):
    languages = stopwords.fileids() # list of supported languages
    stopWords = set(stopwords.words([language for language in languages]))
    
    sentences = []
    for sentence in text.split('.'):
        sentences.append(sentence)
        
    words = word_tokenize(text)
    words = [word for word in words if word not in stopWords]
    
    fdict = FreqDist(words) # frequency distribution
    
    # assign scores to senteces based on word frequencies
    sentence_scores = [sum(fdict[word] for word in word_tokenize(sentence) if word in fdict) for sentence in sentences]
    sentence_scores = list(enumerate(sentence_scores))
    
    # sort descending
    sorted_sentences = sorted(sentence_scores, key = lambda x: x[1], reverse = True)
    
    # Randomly select the top `num_sentences` sentences for the summary
    random_sentences = random.sample(sorted_sentences[:10], num_sen)

    # Sort the randomly selected sentences based on their original order in the text
    summary_sentences = sorted(random_sentences, key=lambda x: x[0])

    # Create the summary
    summary = ' '.join([sentences[i] for i, _ in summary_sentences])

    return summary

In [ ]:
summary = []
for lyrics in list_lyrics_df['lyrics']:
    summary.append(text_summarizer(lyrics))
list_lyrics_df['summary'] = summary
list_lyrics_df

,title,lyrics,negative,neutral,positive,compound,sentiment,lyrics_embed,summary
0,Nếu Ngày Ấy,đã từng hạnh phúc. đã từng đậm sâu. đã từng cù...,0.014,0.986,0.000,-0.5574,Very Negative,"[0.017381342, -0.019343004, -0.02219965, -0.01...",mưa rơi ướt vai cũng là lúc tình ta đã phai
1,MẸ YÊU CON,. . . à à ời. à à ơi. cái cò đi đón cơn mưa. t...,0.000,1.000,0.000,0.0000,Very Negative,"[0.00606683, 0.0065435194, -0.0027725797, -0.0...",thoáng thấy đó hình như bóng dáng bao người
2,Buông,. . . uh uh uh uh uh. la la la la la. . . buon...,0.037,0.946,0.017,-0.9001,Positive,"[-0.017627483, -0.005964813, -0.038477246, -0....",buongiorno a tutt''e nemice ca almeno so cchi...
3,không biết nên vui hay buồn (feat. Táo),. . đâu ai hay ký ức xưa ngỡ như mây. bóng dán...,0.000,0.989,0.011,0.5574,Very Negative,"[-0.028158708, -0.026670555, -0.024711443, -0....",rồi kết thúc tim vẫn nhanh nhưng đỏ mắt
4,Bầu Trời Mới,. . . thức giấc mỗi ngày là khi màn đêm vừa ké...,0.012,0.969,0.019,0.3818,Very Negative,"[0.03414372, -0.0071669323, -0.04063779, -0.01...",giữa muôn vàn vì sao chiếu sáng nơi đây chỉ c...
5,Thế Giới Kẻ Thất Tình,. sau cơn mưa đêm qua. ký ức bỗng ướt nhoà. nh...,0.016,0.984,0.000,-0.6808,Very Negative,"[0.0054905927, -0.0039519547, -0.0600965, -0.0...",yêu em trong tổn thương mối tình đơn phương ấy
6,Cơn Mưa Cuối,lâu nay ta quên nhau chưa. lời ca anh viết vẫn...,0.011,0.983,0.006,-0.6705,Neutral,"[0.026042828, -0.0031009277, -0.032297082, 0.0...",sau bao cố gắng vẫn chỉ một mình anh nơi đây ...
7,Sau Cơn Mưa,. . . vài người nói với em. lòng thủy chung có...,0.010,0.985,0.005,-0.4404,Neutral,"[0.022912256, 0.02659603, -0.005567776, -0.024...",tình yêu sao cứ đến rồi đi thế này
8,Ai Mà Biết Được (feat. tlinh),. . . yeah em đang giấu gì. chẳng buồn nói lí ...,0.004,0.989,0.008,0.2960,Very Negative,"[-0.020836018, -0.008379805, -0.026419071, -0....",vì lần nào thì anh cũng chỉ cần tỏ ra quan tâ...
9,NÉT,. ree ouch hey hm la. ree ouch hey hm la. ree ...,0.107,0.814,0.079,-0.8793,Very Negative,"[0.05491311, 0.03834684, 0.026508601, -0.01724...",ree ree ouch ouch ouch la


In [38]:
mood = "Daydreaming on a snowy day"
lyrics_sample = list_lyrics_df.sample(6)['summary']
mood = mood + "." + ".".join(lyrics_sample)
mood

'Daydreaming on a snowy day. 夢の先へ. a í a thuận nước đẩy thuyền dẫu thác ghềnh sông sâu. một hai ba nước mắt của anh lại rơi. con chỉ mong bố thấy con ở đây nắm tay và hát bố nghe. và sẽ yêu một đời. gonna get it get it together and flower wooh'

In [39]:
mood = mood[:250] if len(mood) > 250 else mood
len(mood)

242

In [40]:
search_recs = sp.search(q = mood, type="track", limit=30)

recs = []
for track in search_recs['tracks']['items']:
    recs.append({
        'name': track['name'],
        'artist': track['artists'][0]['name']
    })
pd.DataFrame(recs)

,name,artist
0,遠舟的少年（《大夢歸離》影視劇趙遠舟·人物曲）,侯明昊
1,Gió (Speed Up),Cukak
2,Mưa Rơi Vào Phòng (Vux Remix),New Media
3,Khi Giấc Mơ Về,Uyên Linh
4,海市蜃楼 - 女声版,一颗狼星
5,潮汐 - DJ版,傅梦彤
6,Bước Qua Mùa Cô Đơn,Vũ.
7,Giấc Mơ Thần Tiên,Forest Studio
8,"Giấc Mơ Tuyết Trắng, Pt. 1",Thủy Tiên
9,Daydreams,JSOL
